<h1>contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#passing-arrays-to-and-from-C++-lib" data-toc-modified-id="passing-arrays-to-and-from-C++-lib-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>passing arrays to and from C++ lib</a></span><ul class="toc-item"><li><span><a href="#WORKING" data-toc-modified-id="WORKING-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>WORKING</a></span></li><li><span><a href="#back-to-python/numpy-array" data-toc-modified-id="back-to-python/numpy-array-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>back to python/numpy array</a></span></li></ul></li><li><span><a href="#start-with-C/C++" data-toc-modified-id="start-with-C/C++-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>start with C/C++</a></span></li><li><span><a href="#resize-an-image" data-toc-modified-id="resize-an-image-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>resize an image</a></span></li><li><span><a href="#determine-image-dimensions-and-channels" data-toc-modified-id="determine-image-dimensions-and-channels-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>determine image dimensions and channels</a></span></li><li><span><a href="#call-C++-code-from-Python" data-toc-modified-id="call-C++-code-from-Python-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>call C++ code from Python</a></span></li></ul></div>

# Building AI from Dirt

## passing arrays to and from C++ lib

C++ code - will not run here

In [ ]:
extern "C"
int *reverse_array(const int *input_array, const int size) {
    int *rev_array = new int[size];
    for (int i = 0; i < size; i++) {
        rev_array[size - i] = input_array[i];
    }
    return rev_array;
}

In [2]:
import numpy as np
import ctypes
from ctypes import *

### WORKING

In [38]:
primes = np.array([2,3,5,7,11])

In [39]:
primes.shape

(5,)

In [42]:
primes.tolist()

[2, 3, 5, 7, 11]

In [11]:
ptr_primes = pointer((ctypes.c_int * 5)(*primes))

In [12]:
reverse_array = cdll.LoadLibrary('./cmake-build-debug/libdirt_lib.dylib').reverse_array
reverse_array.restype = ctypes.POINTER(ctypes.c_int * 5)
result = reverse_array(p_a,5)

In [13]:
print(result.contents)

In [14]:
for p in result.contents:
    print(p)

11
7
5
3
2


### back to python/numpy array

In [35]:
primes.shape

(5,)

In [54]:
np_primes = np.ctypeslib.as_array(result,shape=(1,)).reshape(primes.shape)

In [55]:
np_primes.shape

(5,)

In [56]:
np_primes.tolist()

[11, 7, 5, 3, 2]

## start with C/C++

C++ code - will not run here

In [ ]:
extern "C"
int *split_channels(char *image_file_path) {
    std::string pic_file(image_file_path);
    cv::Mat mat = imread(pic_file, cv::IMREAD_COLOR); // Read the file .c_str()
    std::vector<int> array(mat.rows * mat.cols);
    if (mat.isContinuous()) {
        array.assign(mat.datastart, mat.dataend);
    }
    int *raw_array = &array[0];
    return raw_array;
}

## resize an image

In [82]:
!sips -g pixelHeight -g pixelWidth /Users/Bhill/git/AI_From_Dirt/two.png

/Users/Bhill/git/AI_From_Dirt/two.png
  pixelHeight: 59
  pixelWidth: 46


![tow](two.png)

In [83]:
!cp two.png two.10x10.png ; sips -z 10 10 two.10x10.png

/Users/Bhill/git/AI_From_Dirt/two.10x10.png
  /Users/Bhill/git/AI_From_Dirt/two.10x10.png


![two](two.10x10.png)

## determine image dimensions and channels

In [50]:
h = !identify -format '%h' two.10x10.png
h = int(h[0])
print('h = ' , h)

w = !identify -format '%w' two.10x10.png
w = int(w[0])
print('w = ' , w)

channels = !identify -format '%[channels]' two.10x10.png
has_alpha_channel = channels[0].endswith('a')
print('has 4th alpha channel? ' , has_alpha_channel)

dimension_count =  (3,4) [has_alpha_channel]
print ('total dimesions: ', dimension_count)

h =  10
w =  10
has 4th alpha channel?  True
total dimesions:  4


## call C++ code from Python
**shared** lib from C++ / C, might have dylib extension
**NOTE** you must put .encode() on strings passed to C++/C due to endian encoding
explained in answer at https://stackoverflow.com/questions/52916077/calling-c-function-from-python

In [110]:
!ls cmake-build-debug/

CMakeCache.txt      CTestTestfile.cmake ai_from_dirt.cbp
CMakeFiles          Makefile            cmake_install.cmake


In [56]:
import ctypes
from ctypes import cdll

split_channels = cdll.LoadLibrary('./cmake-build-debug/libdirt_lib.dylib').split_channels
split_channels.restype = ctypes.POINTER(ctypes.c_int * 300)

cpp_array = split_channels("/Users/Bhill/git/AI_From_Dirt/two.10x10.png".encode())
print (cpp_array.contents)
i = 0
for p in cpp_array.contents:
    print(str(i), ' - ', str(p))
    i = i + 1


0  -  240
1  -  237
2  -  235
3  -  255
4  -  242
5  -  241
6  -  238
7  -  255
8  -  246
9  -  243
10  -  240
11  -  255
12  -  246
13  -  244
14  -  243
15  -  255
16  -  246
17  -  246
18  -  244
19  -  255
20  -  244
21  -  247
22  -  242
23  -  255
24  -  246
25  -  248
26  -  241
27  -  255
28  -  245
29  -  244
30  -  241
31  -  255
32  -  240
33  -  238
34  -  236
35  -  255
36  -  238
37  -  237
38  -  233
39  -  255
40  -  243
41  -  242
42  -  241
43  -  255
44  -  228
45  -  237
46  -  241
47  -  255
48  -  197
49  -  223
50  -  240
51  -  255
52  -  172
53  -  214
54  -  240
55  -  255
56  -  134
57  -  185
58  -  241
59  -  255
60  -  139
61  -  141
62  -  239
63  -  255
64  -  158
65  -  140
66  -  237
67  -  255
68  -  209
69  -  215
70  -  241
71  -  255
72  -  243
73  -  243
74  -  241
75  -  255
76  -  241
77  -  242
78  -  240
79  -  255
80  -  247
81  -  246
82  -  243
83  -  255
84  -  151
85  -  201
86  -  239
87  -  255
88  -  88
89  -  138
90  -  239
91  -  255

In [61]:
import numpy as np
from PIL import Image
pixels = np.array(cpp_array.contents).reshape(10,10,3)
pixels[0]

array([[240, 237, 235],
       [255, 242, 241],
       [238, 255, 246],
       [243, 240, 255],
       [246, 244, 243],
       [255, 246, 246],
       [244, 255, 244],
       [247, 242, 255],
       [246, 248, 241],
       [255, 245, 244]], dtype=int32)

In [62]:
mode = 'RGBA'
img = Image.fromarray(pixels,mode)
img.save("output.png")

In [63]:
!identify -format '%[channels]\n' two.10x10.png
!identify two.10x10.png

!identify -format '%[channels]\n' output.png
!identify output.png

srgba
two.10x10.png PNG 10x10 10x10+0+0 8-bit sRGB 1220B 0.000u 0:00.000
srgba
output.png PNG 10x10 10x10+0+0 8-bit sRGB 241B 0.000u 0:00.000


In [64]:
h = !identify -format '%h' two.10x10.png
h = int(h[0])
print('h = ' , h)

w = !identify -format '%w' two.10x10.png
w = int(w[0])
print('w = ' , w)

channels = !identify -format '%[channels]' two.10x10.png
has_alpha_channel = channels[0].endswith('a')
print('has 4th alpha channel? ' , has_alpha_channel)

dimension_count =  (3,4) [has_alpha_channel]
print ('total dimesions: ', dimension_count)

h =  10
w =  10
has 4th alpha channel?  True
total dimesions:  4


![output](output.png)